In [101]:
function kronecker(A, B)
    m, n = size(A)
    p, q = size(B)
    
    result = similar(A, p*m, q*n)
    
    Threads.@threads for A_row in 1:m
        for B_row in 1:p
            for A_col in 1:n
                for B_col in 1:q
                    row = p*(A_row-1) + B_row
                    col = q*(A_col-1) + B_col
                    result[row, col] = A[A_row, A_col] * B[B_row, B_col]
                end
            end
        end
    end
    
    result
end

kronecker (generic function with 1 method)

In [102]:
Threads.nthreads()

8

In [103]:
A = rand(Int, (100, 100)) .% 1000
B = rand(Int, (100, 100)) .% 1000
size(A)

(100, 100)

In [104]:
size(kronecker(A, B))

(10000, 10000)

In [105]:
function power_method(A, max_iterations=100)
    b = rand(eltype(A), size(A)[1])
    
    for _ in 1:max_iterations
        bk = A*b
        # Normalize
        b = bk ./ sum(bk)
    end
    
    b, (A*b)[1] / b[1]
end

power_method (generic function with 2 methods)

In [115]:
A = [3 2; 5 6]
v, l = power_method(A)

([0.28571428571428575, 0.7142857142857143], 7.999999999999998)

In [116]:
v

2-element Vector{Float64}:
 0.28571428571428575
 0.7142857142857143

In [117]:
A*v

2-element Vector{Float64}:
 2.2857142857142856
 5.714285714285714

In [118]:
l*v

2-element Vector{Float64}:
 2.2857142857142856
 5.7142857142857135

In [74]:
function LDU(A)
    L = zeros(eltype(A), size(A))
    D = zeros(eltype(A), size(A))
    U = zeros(eltype(A), size(A))
    
    n, m = size(A)
    @assert n == m
    
    for row in 1:n
        for col in 1:m
            if row < col
                U[row, col] = A[row, col]
            elseif row > col
                L[row, col] = A[row, col]
            else
                D[row, col] = A[row, col]
            end
        end
    end
    
    L, D, U
end

LDU (generic function with 1 method)

In [75]:
L, D, U = LDU(A)
A

2×2 Matrix{Int64}:
 3  2
 5  6

In [76]:
L

2×2 Matrix{Int64}:
 0  0
 5  0

In [77]:
U

2×2 Matrix{Int64}:
 0  2
 0  0

In [78]:
D

2×2 Matrix{Int64}:
 3  0
 0  6

In [79]:
function diag_inverse(D)
    Dinv = zeros(typeof(1 / D[1, 1]), size(D))
    n = size(D)[1]
    
    for i in 1:n
        Dinv[i, i] = 1 / D[i, i]
    end
    
    Dinv
end

diag_inverse (generic function with 1 method)

In [80]:
D * diag_inverse(D)

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

In [99]:
function jacobi(A, b, max_iterations=1000)
    L, D, U = LDU(A)
    Dinv = diag_inverse(D)
    LU = L + U
    
    x = zeros(eltype(A), size(A)[1])
    for _ in 1:max_iterations
        x = Dinv*(b - LU*x)
    end
    
    x
end

jacobi (generic function with 2 methods)

In [100]:
jacobi([1 2; -3 4], [5, 6], 3002)

2-element Vector{Float64}:
 1.644636140577167e264
 4.317169869015055e264